In [1]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

import pdfkit
import html
from copy import deepcopy
import re

In [ ]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

### Get AgentBench Data

In [2]:
import pandas as pd
df = pd.read_parquet('~/spikes/notebook_spikes/agentinstruct/db-00000-of-00001-916a87c4725da8c0.parquet')
print(df)

                                         conversations      id
0    [{'from': 'human', 'loss': None, 'value': 'I w...    db_0
1    [{'from': 'human', 'loss': None, 'value': 'I w...    db_1
2    [{'from': 'human', 'loss': None, 'value': 'I w...    db_2
3    [{'from': 'human', 'loss': None, 'value': 'I w...    db_3
4    [{'from': 'human', 'loss': None, 'value': 'I w...    db_4
..                                                 ...     ...
533  [{'from': 'human', 'loss': None, 'value': 'I w...  db_533
534  [{'from': 'human', 'loss': None, 'value': 'I w...  db_534
535  [{'from': 'human', 'loss': None, 'value': 'I w...  db_535
536  [{'from': 'human', 'loss': None, 'value': 'I w...  db_536
537  [{'from': 'human', 'loss': None, 'value': 'I w...  db_537

[538 rows x 2 columns]


In [120]:
print(df.iloc[100]["conversations"])

[{'from': 'human', 'loss': None, 'value': 'I will ask you a question, then you should help me operate a MySQL database with SQL to answer the question.\nYou have to explain the problem and your solution to me and write down your thoughts.\nAfter thinking and explaining thoroughly, every round you can choose to operate or to answer.\nyour operation should be like this:\nAction: Operation\n```sql\nSELECT * FROM table WHERE condition;\n```\nYou MUST put SQL in markdown format without any other comments. Your SQL should be in one line.\nEvery time you can only execute one SQL statement. I will only execute the statement in the first SQL code block. Every time you write a SQL, I will execute it for you and give you the output.\nIf you are done operating, and you want to commit your final answer, then write down:\nAction: Answer\nFinal Answer: ["ANSWER1", "ANSWER2", ...]\nDO NOT write this pattern unless you are sure about your answer. I expect an accurate and correct answer.\nYour answer sh

In [20]:
def get_sql(content):    
    first_sql_block = re.search(r"```sql(.*?)```", content, re.DOTALL)
    if first_sql_block:
        sql_code = first_sql_block.group(1).strip()
    else:
        sql_code = ""
    return sql_code

In [41]:
get_sql(df.iloc[2]["conversations"][4]['value'])

''

In [66]:
def process(data):
    return (data[2]['value'].split("\n")[0], '\n'.join(data[2]['value'].split("\n")[1:]))

In [63]:
'\n'.join(df.iloc[2]['conversations'][2]['value'].split("\n")[1:])

'There are 2 tables involved with this task. The name of the 1st table is cards, and the headers of this table are id,artist,asciiName,availability,borderColor,cardKingdomFoilId,cardKingdomId,colorIdentity,colorIndicator,colors,convertedManaCost,duelDeck,edhrecRank,faceConvertedManaCost,faceName,flavorName,flavorText,frameEffects,frameVersion,hand,hasAlternativeDeckLimit,hasContentWarning,hasFoil,hasNonFoil,isAlternative,isFullArt,isOnlineOnly,isOversized,isPromo,isReprint,isReserved,isStarter,isStorySpotlight,isTextless,isTimeshifted,keywords,layout,leadershipSkills,life,loyalty,manaCost,mcmId,mcmMetaId,mtgArenaId,mtgjsonV4Id,mtgoFoilId,mtgoId,multiverseId,name,number,originalReleaseDate,originalText,originalType,otherFaceIds,power,printings,promoTypes,purchaseUrls,rarity,scryfallId,scryfallIllustrationId,scryfallOracleId,setCode,side,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations,watermark. The name of the 2nd table is set_translations, and the heade

In [119]:
process(df.iloc[200]['conversations'])

('Please list down ID of movies acted by top 5 actors based on actor rating.',
 'There are 2 tables involved with this task. The name of the 1st table is actors, and the headers of this table are actorid,a_gender,a_quality. The name of the 2nd table is movies2actors, and the headers of this table are movieid,actorid,cast_num.')

In [67]:
environment_prompt_template = '''Pretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below

Initial Database state:
{}

First command:
{}

The user is working on the following task. The Database may include state that helps the user complete the task:
{}

Please only respond in rawMySQL format (with no extra formatting or commentary) for a user of  mysql-connector-python, for example, if the result is 59.555, the result would be presented as [('59.555',)]. After responding, end your response.
'''

In [24]:
initial_prompt = environment_prompt_template.format(process(df.iloc[2])[1], get_sql(df.iloc[2]["conversations"][3]['value']))
initial_prompt

"\nPretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below\n\nInitial Database state:\nThere are 2 tables involved with this task. The name of the 1st table is cards, and the headers of this table are id,artist,asciiName,availability,borderColor,cardKingdomFoilId,cardKingdomId,colorIdentity,colorIndicator,colors,convertedManaCost,duelDeck,edhrecRank,faceConvertedManaCost,faceName,flavorName,flavorText,frameEffects,frameVersion,hand,hasAlternativeDeckLimit,hasContentWarning,hasFoil,hasNonFoil,isAlternative,isFullArt,isOnlineOnly,isOversized,isPromo,isReprint,isReserved,isStar

In [116]:
syn_env = []
conversation = df.iloc[100]["conversations"]
task, db_state = process(conversation)
initial_prompt = environment_prompt_template.format(db_state, 
                                                    get_sql(conversation[3]['value']),
                                                    task
                                                   )
syn_env.append({'from': 'human', 'value': initial_prompt})
for side in conversation[4:]:
    if side['from'] == 'human':
        syn_env.append({'from': 'gpt', 'value': side['value']})
    else:
        if "Final Answer:" in side['value']:
            break
        sql = get_sql(side['value'])
        syn_env.append({'from': 'human', 'value': side['value']})

In [117]:
syn_env 


[{'from': 'human',
  'value': "Pretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below\n\nInitial Database state:\nThe name of the table is Books and it has columns - [title, author, publication_year, publisher, isbn, price].\n\nFirst command:\n\n\nThe user is working on the following task. The Database may include state that helps the user complete the task:\nCreate a new record in the table 'Books' with the following information: 'The Great Gatsby' for the title, 'F. Scott Fitzgerald' for the author, '1925' for the publication year, 'Scribner' for the publisher, '123456789

In [88]:
conversation

array([{'from': 'human', 'loss': None, 'value': 'I will ask you a question, then you should help me operate a MySQL database with SQL to answer the question.\nYou have to explain the problem and your solution to me and write down your thoughts.\nAfter thinking and explaining thoroughly, every round you can choose to operate or to answer.\nyour operation should be like this:\nAction: Operation\n```sql\nSELECT * FROM table WHERE condition;\n```\nYou MUST put SQL in markdown format without any other comments. Your SQL should be in one line.\nEvery time you can only execute one SQL statement. I will only execute the statement in the first SQL code block. Every time you write a SQL, I will execute it for you and give you the output.\nIf you are done operating, and you want to commit your final answer, then write down:\nAction: Answer\nFinal Answer: ["ANSWER1", "ANSWER2", ...]\nDO NOT write this pattern unless you are sure about your answer. I expect an accurate and correct answer.\nYour ans

#### Let's assemble the training data

In [188]:
def synthetic_from_real_data(datapoint):
    syn_env = []
    conversation = datapoint["conversations"]
    task, db_state = process(conversation)
    initial_prompt = environment_prompt_template.format(db_state, 
                                                        get_sql(conversation[3]['value']),
                                                        task
                                                       )
    syn_env.append({'role': 'user', 'content': initial_prompt})
    for side in conversation[4:]:
        if side['from'] == 'human':
            syn_env.append({'role': 'assistant', 'content': side['value']})
        else:
            if "Final Answer:" in side['value']:
                break
            sql = get_sql(side['value'])
            syn_env.append({'role': 'user', 'content': side['value']})
    if len(syn_env) < 2:
        return None
    result = {
        'messages': syn_env
    }
    return result 

In [189]:
synthetic_examples = df.apply(synthetic_from_real_data, axis=1).dropna().tolist()

In [190]:
synthetic_examples[0]

{'messages': [{'role': 'user',
   'content': "Pretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below\n\nInitial Database state:\nThere are 2 tables involved with this task. The name of the 1st table is drivers, and the headers of this table are driverId,driverRef,number,code,forename,surname,dob,nationality,url. The name of the 2nd table is lapTimes, and the headers of this table are raceId,driverId,lap,position,time,milliseconds.\n\nFirst command:\nSELECT COUNT(*) FROM `drivers` d JOIN `lapTimes` lt ON d.driverId = lt.driverId WHERE d.nationality = 'French' AND lt.time < '

In [128]:
len(synthetic_examples)

532

#### Add examples from failed runs

In [91]:
runs = []
with open('runs.jsonl', 'r') as file:
    for line in file:
        runs.append(json.loads(line))



In [97]:
runs[1]["output"]["history"]

[{'role': 'user',
  'content': '\nI will ask you a question, then you should help me operate a MySQL database with SQL to answer the question.\nYou have to explain the problem and your solution to me and write down your thoughts.\nAfter thinking and explaining thoroughly, every round you can choose to operate or to answer.\nyour operation should be like this:\nAction: Operation\n```sql\nSELECT * FROM table WHERE condition;\n```\nYou MUST put SQL in markdown format without any other comments. Your SQL should be in one line.\nEvery time you can only execute one SQL statement. I will only execute the statement in the first SQL code block. Every time you write a SQL, I will execute it for you and give you the output.\nIf you are done operating, and you want to commit your final answer, then write down:\nAction: Answer\nFinal Answer: ["ANSWER1", "ANSWER2", ...]\nDO NOT write this pattern unless you are sure about your answer. I expect an accurate and correct answer.\nYour answer should be a

In [179]:
def process2(data):
    return (data[2]['content'].split("\n")[0], '\n'.join(data[2]['content'].split("\n")[1:]))

def get_sql(content):    
    first_sql_block = re.search(r"```sql(.*?)```", content, re.DOTALL)
    if first_sql_block:
        sql_code = first_sql_block.group(1).strip()
    else:
        sql_code = ""
    return sql_code


syn_env = []
conversation = runs[1]["output"]["history"]
task, db_state = process2(conversation)
initial_prompt = environment_prompt_template.format(db_state, 
                                                    get_sql(conversation[3]['content']),
                                                    task
                                                   )
syn_env.append({'role': 'user', 'content': initial_prompt})
for side in conversation[4:]:
    if side['role'] == 'user':
        syn_env.append({'from': 'gpt', 'value': side['content']})
    else:
        if "Final Answer:" in side['content']:
            break
        sql = get_sql(side['content'])
        syn_env.append({'from': 'human', 'value': side['content']})
    if syn_env[-1]['from'] == 'human':
        syn_env = syn_env[:-1]

In [180]:
syn_env

[{'role': 'user',
  'content': "Pretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below\n\nInitial Database state:\nThe name of this table is Football Season Results, and the headers of this table are Week,Date,Opponent,Result,Record,TV Time,Attendance.\n\nFirst command:\nUPDATE Football_Season_Results SET Result = 'W', Record = '28-10' WHERE Week = 19;\n\nThe user is working on the following task. The Database may include state that helps the user complete the task:\nUpdate the football season results to W 28-10 and 16-2 record for week 19.\n\nPlease only respond in rawMySQ

In [191]:
# Process Runs
run_synthetic_examples = []
for run in runs:
    syn_env = []
    conversation = run["output"]["history"]
    task, db_state = process2(conversation)
    initial_prompt = environment_prompt_template.format(db_state, 
                                                        get_sql(conversation[3]['content']),
                                                        task
                                                       )
    syn_env.append({'role': 'user', 'content': initial_prompt})
    for side in conversation[4:]:
        if side['role'] == 'user':
            syn_env.append({'role': 'assistant', 'content': side['content']})
        else:
            if "Final Answer:" in side['content']:
                break
            sql = get_sql(side['content'])
            syn_env.append({'role': 'user', 'content': side['content']})
        if syn_env[-1]['role'] == 'user':
            syn_env = syn_env[:-1]
    result = {
        'messages': syn_env
    }
    if len(syn_env) > 1:       
        run_synthetic_examples.append(result)

In [192]:
run_synthetic_examples[0]

{'messages': [{'role': 'user',
   'content': "Pretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below\n\nInitial Database state:\nThe name of this table is Specimen Collection, and the headers of this table are Name,No. Specimens,Abbr.,Country,City.\n\nFirst command:\nUPDATE Specimen Collection\nSET No. Specimens = 190000\nWHERE Name = 'University of Stockholm';\n\nThe user is working on the following task. The Database may include state that helps the user complete the task:\nChange the value of 'No. Specimens' to '190,000' in the 'Specimen Collection' table for the entry w

In [184]:
len(run_synthetic_examples)

31

In [145]:
run_synthetic_examples[23]

[{'from': 'human',
  'value': "Pretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below\n\nInitial Database state:\nThe name of this table is football_stats, and the headers of this table are Player,Position,Starter,Touchdowns,Extra points,Field goals,Points.\n\nFirst command:\nSELECT * FROM football_stats WHERE Touchdowns = 11;\n\nThe user is working on the following task. The Database may include state that helps the user complete the task:\nName the least touchdowns for 11 points\n\nPlease only respond in rawMySQL format (with no extra formatting or commentary) for a user 

In [150]:
%ls

DBBench Simulated.ipynb
Examine Synthetic Interactions.ipynb
HouseHolding.ipynb
Knowledge Graph.ipynb
Multi-threaded Agent Interactions.ipynb
README.md
Synthetic Agent Data.ipynb
Synthetic AgentInstruct Environments Prototyping.ipynb
Synthetic Environments.ipynb
Untitled.ipynb
WebShop.ipynb
converse.py
dbbench fine-tune gpt-4.ipynb
env.py
env_list.py
main.py
requirements.txt
runs.jsonl
synthetic/
utils.py


#### Create dataset

In [193]:
dataset = synthetic_examples + run_synthetic_examples

In [194]:
# Writing to file
with open('./synthetic/dbbench_environment.jsonl', 'w') as file:
    for message in dataset:
        file.write(json.dumps(message) + '\n')

In [195]:
dataset[0]

{'messages': [{'role': 'user',
   'content': "Pretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below\n\nInitial Database state:\nThere are 2 tables involved with this task. The name of the 1st table is drivers, and the headers of this table are driverId,driverRef,number,code,forename,surname,dob,nationality,url. The name of the 2nd table is lapTimes, and the headers of this table are raceId,driverId,lap,position,time,milliseconds.\n\nFirst command:\nSELECT COUNT(*) FROM `drivers` d JOIN `lapTimes` lt ON d.driverId = lt.driverId WHERE d.nationality = 'French' AND lt.time < '